### RBF Network Regression

In [8]:
import pandas as pd
from processing_utils import *

raw_dataset = pd.read_csv('./dataset/data.csv')

pruned_dataset = prune_dataset_lines(raw_dataset, remove_nan_lines=False, remove_nan_cols=True)
print("Raw dataset shape =", raw_dataset.shape, " Pruned dataset shape =", pruned_dataset.shape)

encoded_pruned_data = encode_smiles_column_of(pruned_dataset, 'count_encoding') # change to one_hot_encoding here
print("Encoded dataset shape =", encoded_pruned_data.shape)

X_train, y_train, X_val, y_val, X_test, y_test = return_required_data(
    encoded_pruned_data, 
    ['Energy_(kcal/mol)', 'Energy DG:kcal/mol)'], 
    normalize=True
)

Raw dataset shape = (39926, 24)  Pruned dataset shape = (12865, 6)
Encoded dataset shape = (12865, 35)


#### TODO : Use RBF.py to see performance.

In [13]:
%reload_ext autoreload
%autoreload 2
from RBF import RBF
rbf = RBF(n_clusters=8).fit(X_train, y_train)
rbf.predict(X_test)


array([[-0.27220344, -0.27295819],
       [-0.00153994, -0.00145149],
       [-0.00153896, -0.00145051],
       ...,
       [-0.00153993, -0.00145149],
       [-0.00116524, -0.00107735],
       [-0.00138337, -0.00129493]])